# HR Dashboard - Data Cleaning
## Headcount Dataset

This notebook performs data cleaning, wrangling, and feature engineering on the Headcount dataset.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Load Raw Data

In [2]:
# Load the Headcount dataset
df = pd.read_csv('../data/raw/Headcount.csv')
print(f"Original dataset shape: {df.shape}")
df.head(10)

Original dataset shape: (846, 7)


,RecordID,MonthEnd,Department,Headcount,Hires,Terminations,AttritionRate
0,HC-1,2018-01-30,Sales,1,1,0,0.0
1,HC-2,2018-01-30,Marketing,1,1,0,0.0
2,HC-3,2018-01-30,Production,2,2,0,0.0
3,HC-4,2018-01-30,Human Resources,2,2,0,0.0
4,HC-5,2018-01-30,Finance,1,1,0,0.0
5,HC-6,2018-01-30,IT Support,4,4,0,0.0
6,HC-7,2018-01-30,Quality Control,3,3,0,0.0
7,HC-8,2018-01-30,Logistics,2,2,0,0.0
8,HC-9,2018-01-30,Research & Development,6,6,0,0.0
9,HC-10,2018-02-27,Sales,3,2,0,0.0


### Initial Data Exploration

In [3]:
# Display basic information
print("Dataset Info:")
df.info()
print("\n" + "="*50)
print("\nBasic Statistics:")
df.describe()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RecordID       846 non-null    object 
 1   MonthEnd       846 non-null    object 
 2   Department     846 non-null    object 
 3   Headcount      846 non-null    int64  
 4   Hires          846 non-null    int64  
 5   Terminations   846 non-null    int64  
 6   AttritionRate  846 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 46.4+ KB


Basic Statistics:


,Headcount,Hires,Terminations,AttritionRate
count,846.000000,846.000000,846.000000,846.000000
mean,104.784870,2.364066,0.222222,0.002006
std,58.849993,1.515938,0.491613,0.004954
min,1.000000,0.000000,0.000000,0.000000
25%,55.000000,1.000000,0.000000,0.000000
50%,107.000000,2.000000,0.000000,0.000000
75%,157.000000,3.000000,0.000000,0.000000
max,213.000000,8.000000,2.000000,0.047600


In [4]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({'Missing_Count': missing, 'Percentage': missing_percent})
print(missing_df)

Missing Values:
               Missing_Count  Percentage
RecordID                   0         0.0
MonthEnd                   0         0.0
Department                 0         0.0
Headcount                  0         0.0
Hires                      0         0.0
Terminations               0         0.0
AttritionRate              0         0.0


In [5]:
# Check for duplicates
print(f"Total duplicate rows: {df.duplicated().sum()}")

Total duplicate rows: 0


---
## Data Cleaning Steps

### Step 1: Delete Redundant Columns

In [6]:
# Check for redundant columns
print("All columns are relevant for analysis.")
print(f"Columns: {list(df.columns)}")

All columns are relevant for analysis.
Columns: ['RecordID', 'MonthEnd', 'Department', 'Headcount', 'Hires', 'Terminations', 'AttritionRate']


### Step 2: Drop / Rename the Columns

In [7]:
# Rename columns for consistency (snake_case)
df.columns = df.columns.str.lower().str.replace(' ', '_')
print("Renamed columns:")
print(list(df.columns))

Renamed columns:
['recordid', 'monthend', 'department', 'headcount', 'hires', 'terminations', 'attritionrate']


### Step 3: Remove Duplicates

In [8]:
# Remove duplicate rows
print(f"Rows before removing duplicates: {len(df)}")
df = df.drop_duplicates()
print(f"Rows after removing duplicates: {len(df)}")

Rows before removing duplicates: 846
Rows after removing duplicates: 846


### Step 4: Remove the NaN Values from the Dataset

In [9]:
# Check missing values
print("Missing values:")
print(df.isnull().sum())
print(f"\nTotal rows: {len(df)}")

Missing values:
recordid         0
monthend         0
department       0
headcount        0
hires            0
terminations     0
attritionrate    0
dtype: int64

Total rows: 846


In [10]:
# Remove rows with missing critical columns
if df.isnull().sum().sum() > 0:
    df = df.dropna()
    print(f"Rows after removing missing values: {len(df)}")
else:
    print("No missing values found!")

No missing values found!


### Step 5: Clean Individual Columns

#### 5.1 Clean RecordID Column

In [11]:
# Clean RecordID
df['recordid'] = df['recordid'].str.strip()
print("RecordID cleaned")

RecordID cleaned


#### 5.2 Clean MonthEnd Column

In [12]:
# Convert MonthEnd to datetime
df['monthend'] = df['monthend'].str.strip()
df['monthend'] = pd.to_datetime(df['monthend'], errors='coerce')
print(f"MonthEnd converted to datetime")
print(f"Date range: {df['monthend'].min()} to {df['monthend'].max()}")

MonthEnd converted to datetime
Date range: 2018-01-30 00:00:00 to 2025-10-30 00:00:00


In [13]:
# Remove invalid dates
rows_before = len(df)
df = df.dropna(subset=['monthend'])
print(f"Removed {rows_before - len(df)} rows with invalid dates")

Removed 0 rows with invalid dates


In [14]:
# Extract date features
df['year'] = df['monthend'].dt.year
df['month'] = df['monthend'].dt.month
df['quarter'] = df['monthend'].dt.quarter
df['month_name'] = df['monthend'].dt.strftime('%B')
print("Extracted date features")

Extracted date features


#### 5.3 Clean Department Column

In [15]:
# Check Department values
print("Department value counts:")
print(df['department'].value_counts())

Department value counts:
department
Sales                     94
Marketing                 94
Production                94
Human Resources           94
Finance                   94
IT Support                94
Quality Control           94
Logistics                 94
Research & Development    94
Name: count, dtype: int64


In [16]:
# Clean Department
df['department'] = df['department'].str.strip().str.title()
print("\nDepartment cleaned:")
print(df['department'].value_counts())


Department cleaned:
department
Sales                     94
Marketing                 94
Production                94
Human Resources           94
Finance                   94
It Support                94
Quality Control           94
Logistics                 94
Research & Development    94
Name: count, dtype: int64


#### 5.4 Clean Numeric Columns

In [17]:
# Check for negative values
print("Checking for invalid values:")
print(f"Negative Headcount: {(df['headcount'] < 0).sum()}")
print(f"Negative Hires: {(df['hires'] < 0).sum()}")
print(f"Negative Terminations: {(df['terminations'] < 0).sum()}")
print(f"Invalid AttritionRate: {((df['attritionrate'] < 0) | (df['attritionrate'] > 100)).sum()}")

Checking for invalid values:
Negative Headcount: 0
Negative Hires: 0
Negative Terminations: 0
Invalid AttritionRate: 0


In [18]:
# Remove invalid values
df = df[df['headcount'] >= 0]
df = df[df['hires'] >= 0]
df = df[df['terminations'] >= 0]
df = df[(df['attritionrate'] >= 0) & (df['attritionrate'] <= 100)]
print(f"Rows after removing invalid values: {len(df)}")

Rows after removing invalid values: 846


### Step 6: Check for Some More Transformations

#### 6.1 Create Calculated Columns

In [19]:
# Calculate net change in headcount
df['net_change'] = df['hires'] - df['terminations']
print("Created 'net_change' column")

Created 'net_change' column


In [20]:
# Calculate growth rate
df['growth_rate'] = ((df['net_change'] / df['headcount']) * 100).round(2)
print("Created 'growth_rate' column")

Created 'growth_rate' column


In [21]:
# Calculate turnover rate (if not already accurate)
df['turnover_rate'] = ((df['terminations'] / df['headcount']) * 100).round(2)
print("Created 'turnover_rate' column")

Created 'turnover_rate' column


In [22]:
# Calculate hire rate
df['hire_rate'] = ((df['hires'] / df['headcount']) * 100).round(2)
print("Created 'hire_rate' column")

Created 'hire_rate' column


#### 6.2 Create Time-Based Features

In [23]:
# Calculate rolling averages (3-month)
df = df.sort_values(['department', 'monthend'])
df['headcount_3m_avg'] = df.groupby('department')['headcount'].transform(lambda x: x.rolling(3, min_periods=1).mean()).round(0)
df['hires_3m_avg'] = df.groupby('department')['hires'].transform(lambda x: x.rolling(3, min_periods=1).mean()).round(1)
df['terminations_3m_avg'] = df.groupby('department')['terminations'].transform(lambda x: x.rolling(3, min_periods=1).mean()).round(1)
print("Created 3-month rolling averages")

Created 3-month rolling averages


In [24]:
# Calculate month-over-month changes
df['headcount_mom_change'] = df.groupby('department')['headcount'].diff()
df['headcount_mom_change_pct'] = ((df['headcount_mom_change'] / df.groupby('department')['headcount'].shift(1)) * 100).round(2)
print("Created month-over-month change metrics")

Created month-over-month change metrics


#### 6.3 Create Department-Level Features

In [25]:
# Calculate department metrics
dept_stats = df.groupby('department').agg({
    'headcount': ['mean', 'max', 'min'],
    'attritionrate': 'mean',
    'hires': 'sum',
    'terminations': 'sum'
}).round(2)
print("Department statistics:")
print(dept_stats)

Department statistics:
                       headcount          attritionrate hires terminations
                            mean  max min          mean   sum          sum
department                                                                
Finance                    98.29  184   1           0.0   202           18
Human Resources           107.81  209   2           0.0   238           29
It Support                104.93  189   4           0.0   214           25
Logistics                 105.36  205   2           0.0   215           10
Marketing                 103.30  191   1           0.0   212           21
Production                100.94  213   2           0.0   238           25
Quality Control           111.48  205   3           0.0   230           25
Research & Development    112.29  212   6           0.0   232           20
Sales                      98.68  204   1           0.0   219           15


In [26]:
# Add department average headcount
df['dept_avg_headcount'] = df.groupby('department')['headcount'].transform('mean').round(0)
print("\nCreated 'dept_avg_headcount' feature")


Created 'dept_avg_headcount' feature


#### 6.4 Create Categorical Features

In [27]:
# Create headcount size categories
df['headcount_category'] = pd.cut(
    df['headcount'],
    bins=[0, 20, 50, 100, 500],
    labels=['Small (<20)', 'Medium (20-50)', 'Large (50-100)', 'Very Large (100+)']
)
print("Created 'headcount_category' feature")
print("\nHeadcount category distribution:")
print(df['headcount_category'].value_counts().sort_index())

Created 'headcount_category' feature

Headcount category distribution:
headcount_category
Small (<20)           82
Medium (20-50)       115
Large (50-100)       195
Very Large (100+)    454
Name: count, dtype: int64


In [28]:
# Create attrition risk categories
df['attrition_risk'] = pd.cut(
    df['attritionrate'],
    bins=[0, 5, 10, 15, 100],
    labels=['Low (<5%)', 'Medium (5-10%)', 'High (10-15%)', 'Critical (>15%)']
)
print("\nCreated 'attrition_risk' feature")
print("\nAttrition risk distribution:")
print(df['attrition_risk'].value_counts().sort_index())


Created 'attrition_risk' feature

Attrition risk distribution:
attrition_risk
Low (<5%)          159
Medium (5-10%)       0
High (10-15%)        0
Critical (>15%)      0
Name: count, dtype: int64


#### 6.5 Create Binary Flags

In [29]:
# Create flags for analysis
df['has_hires'] = (df['hires'] > 0).astype(int)
df['has_terminations'] = (df['terminations'] > 0).astype(int)
df['is_growing'] = (df['net_change'] > 0).astype(int)
df['is_shrinking'] = (df['net_change'] < 0).astype(int)
print("Created binary flag features")

Created binary flag features


#### 6.6 Sort Data

In [30]:
# Sort by Department and MonthEnd
df = df.sort_values(['department', 'monthend']).reset_index(drop=True)
print("Data sorted by Department and MonthEnd")
df.head(10)

Data sorted by Department and MonthEnd


,recordid,monthend,department,headcount,hires,terminations,attritionrate,year,month,quarter,...,terminations_3m_avg,headcount_mom_change,headcount_mom_change_pct,dept_avg_headcount,headcount_category,attrition_risk,has_hires,has_terminations,is_growing,is_shrinking
0,HC-5,2018-01-30,Finance,1,1,0,0.0,2018,1,1,...,0.0,NaN,NaN,98.0,Small (<20),NaN,1,0,1,0
1,HC-14,2018-02-27,Finance,2,1,0,0.0,2018,2,1,...,0.0,1.0,100.00,98.0,Small (<20),NaN,1,0,1,0
2,HC-23,2018-03-30,Finance,9,7,0,0.0,2018,3,1,...,0.0,7.0,350.00,98.0,Small (<20),NaN,1,0,1,0
3,HC-32,2018-04-29,Finance,11,2,0,0.0,2018,4,2,...,0.0,2.0,22.22,98.0,Small (<20),NaN,1,0,1,0
4,HC-41,2018-05-30,Finance,15,4,0,0.0,2018,5,2,...,0.0,4.0,36.36,98.0,Small (<20),NaN,1,0,1,0
5,HC-50,2018-06-29,Finance,18,3,0,0.0,2018,6,2,...,0.0,3.0,20.00,98.0,Small (<20),NaN,1,0,1,0
6,HC-59,2018-07-30,Finance,20,2,0,0.0,2018,7,3,...,0.0,2.0,11.11,98.0,Small (<20),NaN,1,0,1,0
7,HC-68,2018-08-30,Finance,21,1,0,0.0,2018,8,3,...,0.0,1.0,5.00,98.0,Medium (20-50),NaN,1,0,1,0
8,HC-77,2018-09-29,Finance,21,0,0,0.0,2018,9,3,...,0.0,0.0,0.00,98.0,Medium (20-50),NaN,0,0,0,0
9,HC-86,2018-10-30,Finance,24,3,0,0.0,2018,10,4,...,0.0,3.0,14.29,98.0,Medium (20-50),NaN,1,0,1,0


#### 6.7 Final Data Quality Check

In [31]:
# Final summary
print("=" * 60)
print("FINAL DATA QUALITY SUMMARY")
print("=" * 60)
print(f"\nFinal shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nMissing values:")
print(df.isnull().sum())
print(f"\nDuplicates: {df.duplicated().sum()}")

FINAL DATA QUALITY SUMMARY

Final shape: (846, 27)

Columns: ['recordid', 'monthend', 'department', 'headcount', 'hires', 'terminations', 'attritionrate', 'year', 'month', 'quarter', 'month_name', 'net_change', 'growth_rate', 'turnover_rate', 'hire_rate', 'headcount_3m_avg', 'hires_3m_avg', 'terminations_3m_avg', 'headcount_mom_change', 'headcount_mom_change_pct', 'dept_avg_headcount', 'headcount_category', 'attrition_risk', 'has_hires', 'has_terminations', 'is_growing', 'is_shrinking']

Missing values:
recordid                      0
monthend                      0
department                    0
headcount                     0
hires                         0
terminations                  0
attritionrate                 0
year                          0
month                         0
quarter                       0
month_name                    0
net_change                    0
growth_rate                   0
turnover_rate                 0
hire_rate                     0
headcount_3

### Save Cleaned Data

In [32]:
# Save to processed folder
output_path = '../data/processed/Headcount_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"Cleaned data saved to: {output_path}")
print(f"Total records: {len(df)}")

Cleaned data saved to: ../data/processed/Headcount_cleaned.csv
Total records: 846


---
## Summary

### Data Cleaning & Feature Engineering Completed!

**Steps Performed:**
1. ✅ Checked for redundant columns
2. ✅ Renamed columns to snake_case
3. ✅ Removed duplicates
4. ✅ Handled missing values
5. ✅ Cleaned all columns:
   - RecordID, MonthEnd (datetime)
   - Department (standardized)
   - Numeric columns validated
6. ✅ **Feature Engineering:**
   - **Calculated metrics**: net_change, growth_rate, turnover_rate, hire_rate
   - **Time-based**: rolling averages, month-over-month changes
   - **Department-level**: average headcount
   - **Categories**: headcount size, attrition risk
   - **Binary flags**: has_hires, has_terminations, is_growing, is_shrinking

**Feature Summary:**
- Original columns: 7
- Final columns: 30+
- New features: 23+